In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/carrodosdataset/Carro-DoS-dataset.csv


In [2]:
dados = pd.read_csv("/kaggle/input/carrodosdataset/Carro-DoS-dataset.csv")

In [3]:
dados.columns


Index(['1478198376.389427', '0316', '8', '05', '21', '68', '09', '21.1',
       '21.2', '00', '6f', 'R'],
      dtype='object')

In [4]:
# Atribuíndo rótulos às colunas pra facilitar. Eu indico pra vocês o que é cada coluna. 
# Os nomes estão em inglês pra facilitar a correção com a documentação do dataset.
#    1.    Timestamp : em segundos; CAN ID : identificador da mensagem CAN em Hexadecimal HEX (ex. 043f)
#    3.    DLC : número de bytes de dados de 0 a 8
#    4.    DATA[0~7] : valor do dado (byte)
#    5.    Flag : T or R, T represents injected message while R represents normal message

Columns = ["Timestamp", "CANID", "DLC", "DATA0", "DATA1", "DATA2", "DATA3", "DATA4", "DATA5", "DATA6", "DATA7", "Flag"]

In [5]:
#Agora vejamos novamente após a atribuição dos rótulos nas colunas.
dados = pd.read_csv("/kaggle/input/carrodosdataset/Carro-DoS-dataset.csv", header=None, names = Columns)
dados.columns

Index(['Timestamp', 'CANID', 'DLC', 'DATA0', 'DATA1', 'DATA2', 'DATA3',
       'DATA4', 'DATA5', 'DATA6', 'DATA7', 'Flag'],
      dtype='object')

In [6]:
dados.Flag

0          R
1          R
2          R
3          R
4          R
          ..
3665766    R
3665767    R
3665768    R
3665769    R
3665770    R
Name: Flag, Length: 3665771, dtype: object

In [7]:
dados.Flag = dados.Flag.map({'R':0, 'T':1})

In [8]:
dados.Flag

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
3665766    0.0
3665767    0.0
3665768    0.0
3665769    0.0
3665770    0.0
Name: Flag, Length: 3665771, dtype: float64

In [9]:
dados.Timestamp

0          1.478198e+09
1          1.478198e+09
2          1.478198e+09
3          1.478198e+09
4          1.478198e+09
               ...     
3665766    1.478201e+09
3665767    1.478201e+09
3665768    1.478201e+09
3665769    1.478201e+09
3665770    1.478201e+09
Name: Timestamp, Length: 3665771, dtype: float64

In [10]:
print(dados.isnull().sum())

Timestamp        0
CANID            0
DLC              0
DATA0            0
DATA1            0
DATA2            0
DATA3        31188
DATA4        31188
DATA5        31188
DATA6        31188
DATA7        31188
Flag         31188
dtype: int64


In [11]:
updados = dados.dropna(axis=0)
#y1 = updados['DATA3']
#updados.drop("DATA3",axis=1,inplace=True)
updados.drop("DATA4",axis=1,inplace=True)
updados.drop("DATA5",axis=1,inplace=True)
updados.drop("DATA6",axis=1,inplace=True)
updados.drop("DATA7",axis=1,inplace=True)
print(updados.isnull().sum())

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Timestamp    0
CANID        0
DLC          0
DATA0        0
DATA1        0
DATA2        0
DATA3        0
Flag         0
dtype: int64


In [12]:
#Importando a biblioteca de preprocessamento dos dados
from sklearn import preprocessing

In [13]:
#Antes de treinar o modelo, alguns passos adicionais são necessários para o preprocessamento, pois algumas colunas possuem falores não numéricos.
# sklearn.preprocessing.LabelEncoder codifica os valores entre 0 e n_classes-1 e fit_transform(y). 
# Ajusta os códigos e retorna os valores codificados.
# No nosso caso, nós transformamos valores não numéricos em valores numéricos. 
# Nós precisamos preprocessar quatro classes: protocol_type, service, flag, e label.
# Para fazer isso, nós usamos o método fit.transform(), que calibra as medições. 
#
updados.CANID = preprocessing.LabelEncoder().fit_transform(updados["CANID"])
updados.DATA0 = preprocessing.LabelEncoder().fit_transform(updados["DATA0"])
updados.DATA1 = preprocessing.LabelEncoder().fit_transform(updados["DATA1"])
updados.DATA2 = preprocessing.LabelEncoder().fit_transform(updados["DATA2"])
updados.DATA3 = preprocessing.LabelEncoder().fit_transform(updados["DATA3"])
#updados.DATA4 = preprocessing.LabelEncoder().fit_transform(dados["DATA4"])
#updados.DATA5 = preprocessing.LabelEncoder().fit_transform(dados["DATA5"])
#updados.DATA6 = preprocessing.LabelEncoder().fit_transform(dados["DATA6"])
#updados.DATA7 = preprocessing.LabelEncoder().fit_transform(dados["DATA7"])

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [14]:
#updados.DATA3.values

In [15]:
# Vamos identificar o nosso dado. 
#
Columns1 = ["Timestamp", "CANID", "DLC", "DATA0", "DATA1", "DATA2", "DATA3"] #, "DATA4", "DATA5", "DATA6", "DATA7", "Flag"]\
X = updados[Columns1].values
y = updados.Flag.values
print(y)
print(X)

[0. 0. 0. ... 0. 0. 0.]
[[1.47819838e+09 1.30000000e+01 8.00000000e+00 ... 2.00000000e+00
  3.30000000e+01 2.00000000e+00]
 [1.47819838e+09 8.00000000e+00 8.00000000e+00 ... 2.80000000e+01
  0.00000000e+00 0.00000000e+00]
 [1.47819838e+09 1.00000000e+01 8.00000000e+00 ... 2.00000000e+00
  1.10000000e+01 7.00000000e+00]
 ...
 [1.47820121e+09 1.10000000e+01 8.00000000e+00 ... 0.00000000e+00
  4.70000000e+01 5.00000000e+00]
 [1.47820121e+09 1.40000000e+01 8.00000000e+00 ... 6.10000000e+01
  3.90000000e+01 4.00000000e+00]
 [1.47820121e+09 2.20000000e+01 8.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 2.30000000e+01]]


In [16]:
# Tentando o classificador Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [17]:
# Separando as amostras para o classificador. 
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=101)

In [18]:
# Treinando o modelo de ML para detectar intrusões. 

clf = RandomForestClassifier(max_depth=2, random_state=0)
lr = LogisticRegression()
clf.fit(X_train, y_train)
lr.fit(X_train,y_train)
pred = lr.predict(X_test)
Score1 = clf.score(X_test,y_test)
Score2 = lr.score(X_test,y_test)
print(Score1*100)
print(Score2*100)
print(accuracy_score(pred,y_test))



100.0
83.88609423363522
0.8388609423363522
